In [2]:
from __future__ import print_function

import pandas as pd
import scipy as sp

from abtools.bayesian import ARPPUModel

%matplotlib inline

In [3]:
mu = 9
size = 300
delta_true = .05

arppu_a = pd.Series(sp.stats.wald.rvs(loc=mu, size=size, random_state=8228))
arppu_b = pd.Series(sp.stats.wald.rvs(loc=mu*(1+delta_true), size=size, random_state=8228))

print('ARPPU')
print('Mean: %.4f' % arppu_a.mean())
print('Mean: %.4f' % arppu_b.mean())
print('B больше A на %.4f = %d%%' % (arppu_b.mean()- arppu_a.mean(), 
                                    (arppu_b.mean()/arppu_a.mean()-1)*100))

ARPPU
Mean: 9.9796
Mean: 10.4296
B больше A на 0.4500 = 4%


In [ ]:
model = ARPPUModel(arppu_a, arppu_b)